In [1]:
import tsplib95
import random
from itertools import product

In [2]:
#DATOS DEL PROBLEMA
file = "swiss42.tsp" 
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion(lista de nodos)
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


Algoritmo de colonia de hormigas

La función Add_Nodo selecciona al azar un nodo con probabilidad uniforme.
Para ser mas eficiente debería seleccionar el próximo nodo siguiendo la probabilidad correspondiente a la ecuación:

$p^k_{ij}(t) = \frac{[\tau_{ij}(t)]^\alpha[\nu_{ij}]^\beta}{\sum_{l\in J^k_i} [\tau_{il}(t)]^\alpha[\nu_{il}]^\beta}$, si $j \in J^k_i$

$p^k_{ij}(t) = 0$, si $j \notin J^k_i$

In [3]:
def Add_Nodo(problem, H, T, alpha, beta):
  
  nodos = list(problem.get_nodes())
  # Obtenemos los nodos que no han sido visitados
  nodos_no_visitados = [x for x in nodos if x not in H]
  nodo_actual = H[-1]
  
  probabilidades= []
  sumatorio = 0

  # Para cada nodo no visitado calculamos su probabilidad de ser elegido
  # según las feromonas y la inversa de la distancia desde el nodo actual
  for nodo in nodos_no_visitados:
    t = T[nodo_actual][nodo]
    n = 1/distancia(nodo_actual, nodo, problem)

    probabilidad = (t**alpha)*(n**beta)
    probabilidades.append(probabilidad)
    sumatorio += probabilidad

  # Normalizamos las probabilidades dividiendo por el sumatorio de todas
  probabilidades_normalizadas = [probabilidad/sumatorio for probabilidad in probabilidades]

  # Elegimos un nodo de forma aleatoria según las probabilidades calculadas
  nodo_elegido = random.choices(nodos_no_visitados, weights=probabilidades_normalizadas)[0]

  return nodo_elegido

def Incrementa_Feromona(problem, T, H) :
  #Incrementa segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  #Mejora:Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [4]:
def hormigas(problem, N, alpha, beta) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
  #alpha, beta = parámetros de la ecuación para elegir nodo siguiente

  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges())

  #Inicializa las aristas con una cantidad inicial de feromonas:1
  #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]

  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]

  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :

      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h], T, alpha, beta)
      Hormiga[h].append(Nuevo_Nodo)

    #Incrementa feromonas en esa arista
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)

    #Evapora Feromonas
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual

  return mejor_solucion, mejor_distancia

In [6]:
# Probamos diferentes combinaciones de alpha y beta para encontrar la combinación 
# que mejor solución nos da, proband con valores de 0 a 1 en intervalos de 0.25
numeros = [round(i * 0.25,2) for i in range(int(1 / 0.25) + 1)]
combinaciones = list(product(numeros, repeat=2))
mejor_solucion = []
mejor_distancia = 10e100
mejor_alpha = -1
mejor_beta = -1

for combinacion in combinaciones:
    solucion, distancia_sol = hormigas(problem, 500, combinacion[0], combinacion[1])
    if distancia_sol < mejor_distancia:
        mejor_solucion = solucion
        mejor_distancia = distancia_sol
        mejor_alpha = combinacion[0]
        mejor_beta = combinacion[1]

print("Mejor alpha:", mejor_alpha)
print("Mejor beta:", mejor_beta)

# Para la mejor combinación de alpha y beta, ejecutamos el algoritmo con un mayor número de hormigas
solucion_final, distancia_final = hormigas(problem, 25000, mejor_alpha, mejor_beta)
print("La mejor solución es:", solucion_final)
print("Con una distancia total de:", distancia_final)

Mejor alpha: 0.5
Mejor beta: 1.0
La mejor solución es: [0, 6, 1, 27, 2, 26, 18, 39, 9, 25, 11, 12, 5, 19, 13, 17, 37, 14, 7, 30, 29, 8, 22, 38, 21, 33, 20, 34, 24, 40, 23, 41, 10, 3, 4, 28, 32, 31, 36, 35, 16, 15]
Con una distancia total de: 2330
